# Preprocessing

## Table of contents
* [Overview](#Overview)
* [1. Imports](#1.-Imports)
* [2. Data Load](#2.-Load-data-and-drop-variables-to-run-dummy-variables)
* [3. Dummy Variables](#3.-Dummy-Variables)
* [4. Scale Data](#4.-Scale-Data)
* [5. Concatenate Dataframes](#5.-Concatenate-Dataframes)
* [6. Split train and test](#6.-Split-train-and-test-dataset)

## Overview

1. Pick proper categorical variables to run machine learning model.
2. Create dummy variables.
3. Apply StandardScaler to scale int and float variables.
4. Concat dummy variables, scaled data, and dependent variable ('Churn Label').

## 1. Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve

## 2. Load data and drop variables to run dummy variables

In [2]:
# data load
df = pd.read_pickle('/Users/hansangjun/Desktop/Springboard/Capstone2/telco_data/newdat')

In [3]:
# churn label should be object type.
df['Churn Label'] = df['Churn Label'].astype(object)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   City               7043 non-null   object 
 2   Zip Code           7043 non-null   object 
 3   Gender             7043 non-null   object 
 4   Senior Citizen     7043 non-null   object 
 5   Partner            7043 non-null   object 
 6   Dependents         7043 non-null   object 
 7   Tenure Months      7043 non-null   int64  
 8   Phone Service      7043 non-null   object 
 9   Multiple Lines     7043 non-null   object 
 10  Internet Service   7043 non-null   object 
 11  Online Security    7043 non-null   object 
 12  Online Backup      7043 non-null   object 
 13  Device Protection  7043 non-null   object 
 14  Tech Support       7043 non-null   object 
 15  Streaming TV       7043 non-null   object 
 16  Streaming Movies   7043 

In [5]:
# Drop variables
df.drop(columns=['CustomerID', 'City', 'Zip Code'], inplace=True)

In [6]:
# check the object type
df.select_dtypes(include=[object])

,Gender,Senior Citizen,Partner,Dependents,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Churn Label,Churn Reason
0,Male,No,No,No,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,1,Competitor made better offer
1,Female,No,No,Yes,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,1,Moved
2,Female,No,No,Yes,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,1,Moved
3,Female,No,Yes,Yes,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,1,Moved
4,Male,No,No,Yes,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),1,Competitor had better devices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,No,No,No,Yes,No,No,No,No,No,No,No,No,Two year,Yes,Bank transfer (automatic),0,NaN
7039,Male,No,Yes,Yes,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,0,NaN
7040,Female,No,Yes,Yes,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),0,NaN
7041,Female,No,Yes,Yes,No,No,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,0,NaN


## 3. Dummy Variables
1. extract object type column names.
2. make sure deleting dependent variable.
3. create dummy variables

In [7]:
# Extract object type column names from a new list.
names_dummies = df.select_dtypes(include=['object']).columns.to_list()

# Dependent variable doesn't have to be a dummy variable.
names_dummies.remove('Churn Label')
print(names_dummies)

['Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Churn Reason']


In [8]:
# Create dummy variables.
df = pd.get_dummies(df, columns=names_dummies, prefix=names_dummies)

# Print the columns names
print(df.columns)

Index(['Tenure Months', 'Monthly Charges', 'Total Charges', 'Churn Label',
       'Churn Score', 'CLTV', 'Gender_Female', 'Gender_Male',
       'Senior Citizen_No', 'Senior Citizen_Yes', 'Partner_No', 'Partner_Yes',
       'Dependents_No', 'Dependents_Yes', 'Phone Service_No',
       'Phone Service_Yes', 'Multiple Lines_No', 'Multiple Lines_Yes',
       'Internet Service_DSL', 'Internet Service_Fiber optic',
       'Internet Service_No', 'Online Security_No', 'Online Security_Yes',
       'Online Backup_No', 'Online Backup_Yes', 'Device Protection_No',
       'Device Protection_Yes', 'Tech Support_No', 'Tech Support_Yes',
       'Streaming TV_No', 'Streaming TV_Yes', 'Streaming Movies_No',
       'Streaming Movies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'Paperless Billing_No', 'Paperless Billing_Yes',
       'Payment Method_Bank transfer (automatic)',
       'Payment Method_Credit card (automatic)',
       'Payment Method_Electronic check', 'Pa

In [9]:
df.head()

,Tenure Months,Monthly Charges,Total Charges,Churn Label,Churn Score,CLTV,Gender_Female,Gender_Male,Senior Citizen_No,Senior Citizen_Yes,...,Churn Reason_Lack of self-service on Website,Churn Reason_Limited range of services,Churn Reason_Long distance charges,Churn Reason_Moved,Churn Reason_Network reliability,Churn Reason_Poor expertise of online support,Churn Reason_Poor expertise of phone support,Churn Reason_Price too high,Churn Reason_Product dissatisfaction,Churn Reason_Service dissatisfaction
0,2,53.85,108.15,1,86,3239,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,70.70,151.65,1,67,2701,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,8,99.65,820.50,1,86,5372,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,28,104.80,3046.05,1,84,5003,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,49,103.70,5036.30,1,89,5340,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


Dummy variables has been created.

## 4. Scale Data
1. collect int and float.
2. define scaler.
3. fit scale_df
4. transform the data using fitted scaler.
5. make it dataframe.

In [10]:
# collect int and float
names_list = ['Tenure Months', 'Monthly Charges', 'Total Charges', 'Churn Score', 'CLTV']
names_list_SS = ['Tenure Months_SS', 'Monthly Charges_SS', 'Total Charges_SS', 'Churn Score_SS', 'CLTV_SS']
scale_df = df[names_list]

# define scaler
scaler = StandardScaler()

# fit scale_df
scaler.fit(scale_df)

# transform the data using fitted scaler
scaled_df = scaler.transform(scale_df)

# make it dataframe
scaled_df = pd.DataFrame(scaled_df, columns=names_list_SS) 
scaled_df.head()

,Tenure Months_SS,Monthly Charges_SS,Total Charges_SS,Churn Score_SS,CLTV_SS
0,-1.236724,-0.362660,-0.958066,1.268402,-0.981675
1,-1.236724,0.197365,-0.938874,0.385650,-1.436462
2,-0.992402,1.159546,-0.643789,1.268402,0.821409
3,-0.177995,1.330711,0.338085,1.175481,0.509483
4,0.677133,1.294151,1.216150,1.407784,0.794358


## 5. Concatenate Dataframes

In [11]:
# drop original int and float varaibles
df.drop(columns=names_list, inplace=True)

# concatenating scaled_df and df along columns
df = pd.concat([scaled_df, df], axis=1)

In [12]:
df.head()

,Tenure Months_SS,Monthly Charges_SS,Total Charges_SS,Churn Score_SS,CLTV_SS,Churn Label,Gender_Female,Gender_Male,Senior Citizen_No,Senior Citizen_Yes,...,Churn Reason_Lack of self-service on Website,Churn Reason_Limited range of services,Churn Reason_Long distance charges,Churn Reason_Moved,Churn Reason_Network reliability,Churn Reason_Poor expertise of online support,Churn Reason_Poor expertise of phone support,Churn Reason_Price too high,Churn Reason_Product dissatisfaction,Churn Reason_Service dissatisfaction
0,-1.236724,-0.362660,-0.958066,1.268402,-0.981675,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-1.236724,0.197365,-0.938874,0.385650,-1.436462,1,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,-0.992402,1.159546,-0.643789,1.268402,0.821409,1,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,-0.177995,1.330711,0.338085,1.175481,0.509483,1,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,0.677133,1.294151,1.216150,1.407784,0.794358,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 62 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Tenure Months_SS                                        7043 non-null   float64
 1   Monthly Charges_SS                                      7043 non-null   float64
 2   Total Charges_SS                                        7043 non-null   float64
 3   Churn Score_SS                                          7043 non-null   float64
 4   CLTV_SS                                                 7043 non-null   float64
 5   Churn Label                                             7043 non-null   object 
 6   Gender_Female                                           7043 non-null   uint8  
 7   Gender_Male                                             7043 non-null   uint8  
 8   Senior Citizen_No                     

info() presents scaled data and dummy variables with a dependent variable.

In [17]:
df.to_pickle('df_preprocessed.pkl')

## 6. Split train and test dataset

In [14]:
X = df.loc[:, df.columns != 'Churn Label']
y = df.loc[:, df.columns == 'Churn Label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [15]:
# check the dimension of the splited dataset
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4930, 61)
(2113, 61)
(4930, 1)
(2113, 1)
